### Check an endpoint
This notebook aims to run the pipeline on a given endpoint to check to see if it will be successful. This includes collecting, pipeline and datset stages. It aims to also highlight useful information as a summary as to whether the endpoint would be successful on our platform. it will download all relevant data to do this and hence might be disk intensive. You'll need to provide the following information:
- collection - this is the collection that the dataset belongs to, this can be extracted from the specification but for this notebook we ask to provide it incase you want to test the pipeline on something which isn't being included in the main site right now
- dataset - this is the dataset that the endpoint is meant to provide data for, technically this can be multiple datasets but this this use case it should just be one. It is also the name of the pipeline that is ran on the individual resources that are downloaded from the endpoint. the terms dataset/pipline are often the same
- organisation - the organisation identifier to be used for the endpoint
- endpoint url - the actual url needed for the endpoint
- plugin - often we use plugins to download the data this is only needed for specific endpoints

If you are seeing errors regarding digital-land, then try 

`pip install -e git+https://github.com/digital-land/pipeline.git#egg=digital-land`

And restart the notebook.

In [2]:
import os, time
import pandas as pd
from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource
from digital_land.collection import Collection

/Users/jimmytidey/Projects/jupyter-analysis/venv/lib/python3.10/site-packages/pyproj/transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=uk_os_OSTN15_NTv2_OSGBtoETRS.tif, full_name=, package_name=, url=https://cdn.proj.org/uk_os_OSTN15_NTv2_OSGBtoETRS.tif, direct_download=True, open_license=True, available=False)
  super().__init__(


In [3]:
collection_name = 'listed-building-collection'
dataset = 'listed-building-outline'
organisation = 'local-authority-eng:CAM'
endpoint_url = 'https://raw.githubusercontent.com/jimmytidey/jupyter-analysis/feature-endpoint-checker-api/endpoint_checker/examples/conservation_area_invalid_csv.csv'
plugin = None
additional_column_mappings=None
additional_concats=None
data_dir = 'data/endpoint_checker_' + str(int(time.time()*10000)) 


run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir,
    additional_col_mappings=additional_column_mappings,
    additional_concats=additional_concats
)

collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
http_status = logs['status'].values[0]
print(f'http_status {http_status}')

dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues()
issues = results.loc[(results.issue_type != 'default-value')]
display(issues)


running endpoint workflow
HTTP Error 404: Not Found
HTTP Error 404: Not Found
data/endpoint_checker_16967525553823/collection
[]
*No resources collected view collection logs for more info*
http_status 200


OperationalError: no such table: issue

In [5]:
#southwark convservation area endpoint
# collection_name = 'conservation-area-collection'
# dataset = 'conservation-area'
# organisation = 'local-authority-eng:SWK'
# endpoint_url = 'https://www.southwark.gov.uk/assets/attach/194104/Conservation-Areas.gpkg'
# plugin = None
# additional_column_mappings=None
# additional_concats=None

# doncaster tpos
# collection_name = 'tree-preservation-order-collection'
# dataset = 'tree-preservation-zone'
# organisation = 'local-authority-eng:DNC'
# endpoint_url='https://maps.doncaster.gov.uk/server/rest/services/Planning/TPO_Map/MapServer/1'
# plugin = 'arcgis'
# additional_column_mappings=None
# additional_concats=None

# 
collection_name = 'listed-building-collection'
dataset = 'listed-building-outline'
organisation = 'local-authority-eng:CAM'
endpoint_url = 'https://raw.githubusercontent.com/jimmytidey/jupyter-analysis/feature-endpoint-checker-api/endpoint_checker/examples/conservation_area_no_errors.csv'
plugin = None

additional_column_mappings=None
additional_concats=None

# generic data_dir setting
data_dir = '../data/endpoint_checker'

# example playing with additional confiigs
# additional_concats = [{
#     'dataset':'tree-preservation-zone',
#     'endpoint':'de1eb90a8b037292ef8ae14bfabd1184847ef99b7c6296bb6e75379e6c1f9572',
#     'resource':'e6b0ccaf9b50a7f57a543484fd291dbe43f52a6231b681c3a7cc5e35a6aba254',
#     'field':'reference',
#     'fields':'REFVAL;LABEL',
#     'separator':'/'
# }]


In [6]:
val = run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir,
    additional_col_mappings=additional_column_mappings,
    additional_concats=additional_concats
)

print(val)

running endpoint workflow
HTTP Error 404: Not Found
HTTP Error 404: Not Found
../data/endpoint_checker/collection
[{'resource': '14ef21dcac2e2c55bc22f8ced1f2da457dff6e771d09df7e68ad05f2148f82e7', 'bytes': '397', 'endpoints': '88ff80ad7fc684d599de58126c8742b3a4042fff09851db28f8c051439309dc6', 'organisations': 'local-authority-eng:CAM', 'datasets': 'listed-building-outline', 'start-date': '2023-10-08', 'end-date': ''}]


None


In [33]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues()
issues = results.loc[(results.issue_type != 'default-value')]
display(issues)

OperationalError: no such table: issue

#### Collection log summaries

We need to establish if a resource was downloaded from the endpoint and whether there were any issues during the collection process. Examine the output of the below. There should be one log for the attempt made at downloading from the endpoint. If status code is 200 then the resource was downloaded successfully

In [34]:
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))

collection.resource.records

{}

In [35]:

logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
logs

,bytes,content-type,elapsed,endpoint,resource,status,entry-date,start-date,end-date,exception
0,0,text/plain; charset=utf-8,0.036,a3ca58f66b858db870967258a84d5dbc1e6649fcc73070...,,200,2023-10-06T16:22:05.614399,,,


### Check unnassigned entiities
This process automatically aims to detect and assign entity numbers where entries are currently unnassigned. Examine the list below to see what (if any) entities have been assigned. if you were to include these in an actual pipeline you would need to update the configuration lookup.csv with these values. It's worth checking they are sensible before this happens 

In [6]:
unassigned_entries = pd.read_csv(os.path.join(data_dir,'var','cache','unassigned-entries.csv'))
if len(unassigned_entries) == 0:
    print('no additional entity numbers where required')
else:
    print(unassigned_entries)

                 organisation                   prefix reference
0     local-authority-eng:CAM  listed-building-outline    LB1859
1     local-authority-eng:CAM  listed-building-outline    LB1481
2     local-authority-eng:CAM  listed-building-outline    LB1872
3     local-authority-eng:CAM  listed-building-outline    LB1531
4     local-authority-eng:CAM  listed-building-outline    LB1532
...                       ...                      ...       ...
1956  local-authority-eng:CAM  listed-building-outline     LB117
1957  local-authority-eng:CAM  listed-building-outline    LB1648
1958  local-authority-eng:CAM  listed-building-outline    LB1854
1959  local-authority-eng:CAM  listed-building-outline    LB1724
1960  local-authority-eng:CAM  listed-building-outline    LB1958

[1961 rows x 3 columns]


#### Check logs collated from the pipeline process
We need to read the logs and examine to see if the data points were all read in correctly. This uses the sqlite database to do so with some custom queries. You could directly examine the csvs if the pipeline fails.

First, check the column mappings to see what columns the pipeline automatically mapped. Tf this is empty or missing values, then it's likely to be the reason data isn't appearing at the end.

In [13]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_column_mappings()

results

,end_date,entry_date,field,dataset,start_date,resource,column
0,,2023-10-06T15:57:55Z,end-date,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,end-date
1,,2023-10-06T15:57:55Z,entry-date,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,entry-date
2,,2023-10-06T15:57:55Z,geometry,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,geometry
3,,2023-10-06T15:57:55Z,listed-building,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,listed-building
4,,2023-10-06T15:57:55Z,listed-building-grade,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,listed-building-grade
5,,2023-10-06T15:57:55Z,name,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,name
6,,2023-10-06T15:57:55Z,notes,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,notes
7,,2023-10-06T15:57:55Z,reference,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,reference
8,,2023-10-06T15:57:55Z,start-date,listed-building-outline,,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,start-date


examine the issues logs, we'll look at the types of errors being raised and list all of them. This could be improved in the future by examining the severity

In [14]:

results = dataset_db.get_issues_by_type()
results

,issue_type,count
0,default-value,3922
1,invalid geometry,6


In [18]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues()
results.loc[(results.issue_type != 'default-value') ]
issues = results.loc[(results.issue_type != 'default-value')]
display(issues)


,end_date,entry_date,entry_number,field,issue_type,line_number,dataset,resource,start_date,value
136,,,69,geometry,invalid geometry,70,listed-building-outline,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,,Self-intersection[-0.107009972556572 51.518187...
427,,,214,geometry,invalid geometry,215,listed-building-outline,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,,Ring Self-intersection[-0.119518 51.525822]
896,,,448,geometry,invalid geometry,449,listed-building-outline,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,,Ring Self-intersection[-0.127893 51.518481]
1389,,,694,geometry,invalid geometry,695,listed-building-outline,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,,Too few points in geometry component[-0.183758...
2222,,,1110,geometry,invalid geometry,1111,listed-building-outline,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,,Ring Self-intersection[-0.128844 51.526229]
3201,,,1599,geometry,invalid geometry,1600,listed-building-outline,222aa207c5ad94d02272ac94ac4d0ea93f37a733dd217a...,,Self-intersection[-0.145896 51.540893]


#### Final dataset comparison against the sqlite database

Below are two tables which show the difference betwen what was provided to us and what is ucrrently in the entity table. It is important to bear in mind that we assign entities automaticallyis process, the table above shows what we have added.

In [ ]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_entities()
results.head(26)

In [ ]:
# load in raw resources
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
resources = collection.resource.entries
resources

In [ ]:
# currently this just reads in the raw resource but in  the future this should check for 
# converted resource first
resource = resources[0]['resource']
resource_path = os.path.join(data_dir,'collection','resource',resource)
try:
    raw_resource = pd.read_csv(resource_path)
except (UnicodeDecodeError,TypeError,pd.errors.ParserError):
    converted_resource_dir = os.path.join(data_dir,'var','converted_resources')
    converted_resource_path = os.path.join(converted_resource_dir,f'{resource}.csv') 
    if not os.path.exists(converted_resource_path):
        convert_resource(resource,resource_path,converted_resource_dir,dataset)
    raw_resource = pd.read_csv(converted_resource_path)
    

In [ ]:
raw_resource